# GillesPy2 -> Tellurium training dataset generation

In this notebook, we use StochNetV2's dataset generation process as the target for Tellurium's dataset generation process.

Since the complexity of StochNetV2's implementation is rather high, and at this stage we simply aim to be able to use Tellurium's stochastic simulation trajectories as input for StochNetV2 models, it is enough to somehow craft a bridge between the two instead of modifying StochNetV2's code to accept Tellurium's data format.

## StochNetV2's GillesPy2 generation

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from stochnet_v2.dataset.simulation_gillespy import build_simulation_dataset
from stochnet_v2.utils.file_organisation import ProjectFileExplorer

import tellurium as te

# other
from pathlib import Path
import numpy as np
from importlib import import_module
import pandas as pd

In [87]:
# StochNetV2 dataset configuration
name = "SIR"
n_species = 3
params = ["beta", "gamma"]

model_name = name
nb_settings = 10
nb_trajectories = 10
timestep = 0.1
endtime = 1.0
dataset_id = name

project_folder = Path("").parent.resolve() / model_name
project_explorer = ProjectFileExplorer(project_folder)
dataset_explorer = project_explorer.get_dataset_file_explorer(timestep, dataset_id)

In [88]:
# Generate and save initial settings
CRN_module = import_module(model_name)
CRN_class = getattr(CRN_module, model_name)

settings = CRN_class.get_initial_settings(nb_settings)
np.save(dataset_explorer.settings_fp, settings)

In [89]:
# StochNetV2 dataset generation
dataset = build_simulation_dataset(
    model_name,
    nb_settings,
    nb_trajectories,
    timestep,
    endtime,
    dataset_explorer.dataset_folder,
    params_to_randomize=params,
    how="concat",
)

2023-07-08 11:05:07,954 - GillesPy2 - WARNING - 
            Model.set_parameter has been deprecated.  Future releases of GillesPy2 may
            not support this feature.  Parameter.expression should only be set in the constructor.
            
2023-07-08 11:05:07,955 - GillesPy2 - WARNING - 
            Model.set_parameter has been deprecated.  Future releases of GillesPy2 may
            not support this feature.  Parameter.expression should only be set in the constructor.
            
2023-07-08 11:05:10,274 - GillesPy2 - WARNING - 
            Model.set_parameter has been deprecated.  Future releases of GillesPy2 may
            not support this feature.  Parameter.expression should only be set in the constructor.
            
2023-07-08 11:05:10,276 - GillesPy2 - WARNING - 
            Model.set_parameter has been deprecated.  Future releases of GillesPy2 may
            not support this feature.  Parameter.expression should only be set in the constructor.
            
2023-07-

In [90]:
print(f"shape: {dataset.shape}")
print("- 4: 2 initial settings (1 per parameter), 2 trajectories (1 per initial setting)")
print("- 11: time steps from 0.0 to 1.0 with step size 0.1")
print("- 6: time + 3 species + 2 parameters")

shape: (100, 11, 6)
- 4: 2 initial settings (1 per parameter), 2 trajectories (1 per initial setting)
- 11: time steps from 0.0 to 1.0 with step size 0.1
- 6: time + 3 species + 2 parameters


In [91]:
print("two initial settings, one trajectory for each.")
print("note the randomised parameter values - one parameter randomised per initial setting:\n")
for block in dataset[:4]:
    df = pd.DataFrame(block, columns=["time", "S", "I", "R", "beta", "gamma"])
    print(df, "\n")

two initial settings, one trajectory for each.
note the randomised parameter values - one parameter randomised per initial setting:

    time     S      I     R      beta  gamma
0    0.0  68.0  104.0  80.0  2.952539    1.0
1    0.1  66.0  109.0  77.0  2.952539    1.0
2    0.2  68.0  116.0  68.0  2.952539    1.0
3    0.3  66.0  126.0  60.0  2.952539    1.0
4    0.4  61.0  132.0  59.0  2.952539    1.0
5    0.5  60.0  135.0  57.0  2.952539    1.0
6    0.6  61.0  139.0  52.0  2.952539    1.0
7    0.7  58.0  145.0  49.0  2.952539    1.0
8    0.8  53.0  152.0  47.0  2.952539    1.0
9    0.9  54.0  154.0  44.0  2.952539    1.0
10   1.0  48.0  162.0  42.0  2.952539    1.0 

    time     S      I     R      beta  gamma
0    0.0  68.0  104.0  80.0  2.952539    1.0
1    0.1  70.0  109.0  73.0  2.952539    1.0
2    0.2  67.0  118.0  67.0  2.952539    1.0
3    0.3  70.0  121.0  61.0  2.952539    1.0
4    0.4  66.0  129.0  57.0  2.952539    1.0
5    0.5  70.0  133.0  49.0  2.952539    1.0
6    0.6  

## Tellurium's Gillespie implementation

In [92]:
# Load SIR model from SBML
model = te.loadSBMLModel("example.xml")

In [93]:
# Configure simulator
model.integrator = "gillespie"
model.integrator.seed = 42

In [94]:
def randomize_parameters(model, n=1, sigma=0.1):
    """
    Returns: list of n dictionaries, where keys are parameters.
    """
    parameter_names = model.getGlobalParameterIds()
    parameter_values = model.getGlobalParameterValues()

    random_parameters = []
    num_parameters = len(parameter_names)

    for i in range(n):
        iteration_parameters = {}

        for j, (name, value) in enumerate(zip(parameter_names, parameter_values)):
            if i % num_parameters == j:
                shift = np.random.uniform(-sigma, sigma) * value
                iteration_parameters[name] = value + shift
            else:
                # keep the default value for other parameters
                iteration_parameters[name] = value

        random_parameters.append(iteration_parameters)

    return random_parameters


def randomize_species_concentrations(model, n=1):
    """
    Returns: list of n dictionaries, where keys are species.
    """
    species_names = model.getFloatingSpeciesConcentrationIds()
    species_values = model.getFloatingSpeciesConcentrations()

    random_concentrations = []

    for _ in range(n):
        iteration_concentrations = {}

        for name, value in zip(species_names, species_values):
            low = max(0, int(value / 2))  # at least 0, since concentrations are (usually) whole numbers
            high = int(value * 2)
            iteration_concentrations[name] = np.random.randint(low, high)

        random_concentrations.append(iteration_concentrations)

    return random_concentrations


def assign_custom_values(model, value_dict):
    """
    value_dict: can be species concentrations, parameters values, etc.
    Returns: the model with assigned values.
    """
    for prop_name, prop_value in value_dict.items():
        model[prop_name] = prop_value
        
    return model


def get_variable_names(model):
    names = ["time"]
    names.extend(model.getFloatingSpeciesConcentrationIds())
    names.extend(model.getGlobalParameterIds())
    
    return names

In [95]:
# Simulation configuration to mimic StochNetV2's
n_initial_settings = 10
simulations_per_setting = 10
steps = 11
end_time = 1.0

In [96]:
# generate random initial concentrations and parameter variations
init_concentrations = randomize_species_concentrations(model, n_initial_settings)
randomized_parameters = randomize_parameters(model, n_initial_settings, 0.2)
variable_names = get_variable_names(model)

In [97]:
results = []
for init_setting in range(n_initial_settings):
    for sim_iteration in range(simulations_per_setting):
        model.reset()
        model = assign_custom_values(model, init_concentrations[init_setting])
        model = assign_custom_values(model, randomized_parameters[init_setting])
        sim = model.simulate(0.0, end_time, steps)
        
        # Add the randomized parameters as new columns
        for param_name, param_value in randomized_parameters[init_setting].items():
            param_column = np.full((sim.shape[0], 1), param_value)
            sim = np.hstack((sim, param_column))

        results.append(sim)  # append the numpy array to results
        
te_dataset = np.concatenate([np.expand_dims(a, axis=0) for a in results], axis=0)

...simulating initial setting 0, simulation 0
...simulating initial setting 0, simulation 1
...simulating initial setting 0, simulation 2
...simulating initial setting 0, simulation 3
...simulating initial setting 0, simulation 4
...simulating initial setting 0, simulation 5
...simulating initial setting 0, simulation 6
...simulating initial setting 0, simulation 7
...simulating initial setting 0, simulation 8
...simulating initial setting 0, simulation 9
...simulating initial setting 1, simulation 0
...simulating initial setting 1, simulation 1
...simulating initial setting 1, simulation 2
...simulating initial setting 1, simulation 3
...simulating initial setting 1, simulation 4
...simulating initial setting 1, simulation 5
...simulating initial setting 1, simulation 6
...simulating initial setting 1, simulation 7
...simulating initial setting 1, simulation 8
...simulating initial setting 1, simulation 9
...simulating initial setting 2, simulation 0
...simulating initial setting 2, s

In [98]:
print("two initial settings, one trajectory for each.")
print("note the randomised parameter values - one parameter randomised per initial setting:\n")
for block in te_dataset[:4]:
    df = pd.DataFrame(block, columns=variable_names)
    print(df, "\n")

two initial settings, one trajectory for each.
note the randomised parameter values - one parameter randomised per initial setting:

    time   [I]    [R]    [S]      beta  gamma
0    0.0  62.0   72.0  152.0  2.796481    1.0
1    0.1  64.0   78.0  144.0  2.796481    1.0
2    0.2  68.0   83.0  135.0  2.796481    1.0
3    0.3  70.0   88.0  128.0  2.796481    1.0
4    0.4  71.0   96.0  119.0  2.796481    1.0
5    0.5  74.0  100.0  112.0  2.796481    1.0
6    0.6  68.0  111.0  107.0  2.796481    1.0
7    0.7  70.0  115.0  101.0  2.796481    1.0
8    0.8  68.0  124.0   94.0  2.796481    1.0
9    0.9  72.0  128.0   86.0  2.796481    1.0
10   1.0  76.0  133.0   77.0  2.796481    1.0 

    time   [I]    [R]    [S]      beta  gamma
0    0.0  62.0   72.0  152.0  2.796481    1.0
1    0.1  67.0   78.0  141.0  2.796481    1.0
2    0.2  67.0   83.0  136.0  2.796481    1.0
3    0.3  68.0   90.0  128.0  2.796481    1.0
4    0.4  61.0  101.0  124.0  2.796481    1.0
5    0.5  67.0  103.0  116.0  2.79648

## Equivalence comparison

In [99]:
print(f"GillesPy2 dataset shape: {dataset.shape}")
print(f"Tellurium dataset shape: {te_dataset.shape}")

GillesPy2 dataset shape: (100, 11, 6)
Tellurium dataset shape: (100, 11, 6)


In [100]:
print("GillesPy2 first block:")
print(dataset[1], "\n")

print("Tellurium first block:")
print(te_dataset[1], "\n")

GillesPy2 first block:
[[  0.          68.         104.          80.           2.95253891
    1.        ]
 [  0.1         70.         109.          73.           2.95253891
    1.        ]
 [  0.2         67.         118.          67.           2.95253891
    1.        ]
 [  0.3         70.         121.          61.           2.95253891
    1.        ]
 [  0.4         66.         129.          57.           2.95253891
    1.        ]
 [  0.5         70.         133.          49.           2.95253891
    1.        ]
 [  0.6         66.         141.          45.           2.95253891
    1.        ]
 [  0.7         61.         150.          41.           2.95253891
    1.        ]
 [  0.8         55.         157.          40.           2.95253891
    1.        ]
 [  0.9         52.         162.          38.           2.95253891
    1.        ]
 [  1.          46.         169.          37.           2.95253891
    1.        ]] 

Tellurium first block:
[[  0.          62.          72.      

In [101]:
np.save("gp_dataset", dataset)
np.save("te_dataset", te_dataset)